In [1]:
from __future__ import print_function
# import argparse
import json
import pprint
import requests
# import sys
import urllib
import os
# import math, re, types, datetime, random, time, unicodedata, optparse
from datetime import timedelta
import datetime
# from urllib.error import HTTPError
# from urllib.parse import quote
# from urllib.parse import urlencode
import pandas as pd
import sqlite3
from pandas.io import sql
from pandas.io.json import json_normalize

In [2]:
begmonth=datetime.datetime.now()
# begmonth=begmonth.replace(day=1)
date=begmonth.strftime("%Y-%m-%d %H:%M:%S")
print(date)

2018-02-22 01:23:42


In [3]:
# debug=False
# delay=120

# Yelp API

In [4]:
app_id = 'mrsvBGDT1QDCWnCyW2E6rg'
app_secret = 'ZNhzA0Lj1t52FuLLphqASaBz3V08dFkfWqeiDXCkozLg6e2nNFAKJ66hxEaXNxIi'
data = {'grant_type': 'client_credentials',
        'client_id': app_id,
        'client_secret': app_secret}
token = requests.post('https://api.yelp.com/oauth2/token', data=data)
access_token = token.json()['access_token']
url = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % access_token}

# Yelp v3 API: https://nz.yelp.com/developers/documentation/v3   https://www.yelp.com/developers/documentation/v3/business_search

params = {'location': 'Boston, MA', # for loop of zipcodes in Boston and Cambridge
          'categories': 'active', 
          # active stand for all businesses https://www.yelp.com/developers/documentation/v3/all_category_list
          'limit': 50} # maximum 50
response = requests.get(url=url, params=params, headers=headers)

results = response.json()['businesses']
for business in results:
     print(business['name'])

print ('\nTotal Businesses retrieved:' , len(results))

Boston Common
Trapology Boston
New England Aquarium
Urban AdvenTours
Btone Fitness - Back Bay
The Esplanade
South Boston Yoga
Christopher Columbus Waterfront Park
B/SPOKE Indoor Cycling Studio
YogaWorks Back Bay
Room Escapers - Boston Escape Rooms
Norman B Leventhal Park
Charles River Bike Paths
Escape The Room Boston
Healthworks Fitness Center
YogaWorks Comm Ave
Peter Welch's Gym
The Lawn On D
Exhale Back Bay
BURN Fitness Studios
Diablo Glass School
HarborWalk
North End Yoga
Kings Dining & Entertainment - Boston Back Bay
Freedom Trail Run
Boston Marathon
The Charles River
New England Aquarium Whale Watch
Swet Studio
Commonwealth Sports Club
Community Boating
Joint Ventures Physical Therapy and Fitness
Flywheel Sports - Back Bay
Arnold Arboretum
Sadhana Yoga Boston
Cambridge Center Roof Garden
Fort Independence Park
Georges Island
Shakespeare on the Common
Barry's Bootcamp
EverybodyFights
Peters Park
The Boston Common Frog Pond
Rose Fitzgerald Kennedy Greenway
Dance in Boston - Michele

In [5]:
# Boston zipcodes: http://www.city-data.com/zipmaps/Boston-Massachusetts.html
# Cambridge zipcodes: http://www2.cambridgema.gov/CityOfCambridge_Content/documents/ZipCodeMap.pdf

zipcodes = ["02108", "02109", "02110", "02111", "02113", "02114", "02115", "02116", "02118", "02119", "02120", 
            "02121", "02122", "02124", "02125", "02126", "02127", "02128", "02129", "02130", "02131", "02132", 
            "02134", "02135", "02136", "02151", "02152", "02163", "02199", "02203", "02210", "02215", "02467", 
            "02138", "02139", "02140", "02141", "02142"]

In [6]:
app_id = 'mrsvBGDT1QDCWnCyW2E6rg'
app_secret = 'ZNhzA0Lj1t52FuLLphqASaBz3V08dFkfWqeiDXCkozLg6e2nNFAKJ66hxEaXNxIi'
data = {'grant_type': 'client_credentials',
        'client_id': app_id,
        'client_secret': app_secret}
token = requests.post('https://api.yelp.com/oauth2/token', data=data)
access_token = token.json()['access_token']
url = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % access_token}

# Yelp v3 API: https://nz.yelp.com/developers/documentation/v3   
#https://www.yelp.com/developers/documentation/v3/business_search

for z in zipcodes:   
#     zipcode = query_api("Businesses in zipcodes: ", z)
    params = {'location': '%s' % z, 
              # for loop of zipcodes in Boston and Cambridge
              #%s means replaced by zipcode
              'categories': 'active', # active stands for all businesses https://www.yelp.com/developers/documentation/v3/all_category_list
              'limit': 50} # maximum 50
    
response = requests.get(url=url, params=params, headers=headers)

results = response.json()['businesses']
for business in results:
     print(business['name'], business['location'])

print ('\nTotal Businesses retrieved:', len(results))

Cambridge Center Roof Garden {'address1': '4 Cambridge Ctr', 'address2': '', 'address3': '', 'city': 'Cambridge', 'zip_code': '02142', 'country': 'US', 'state': 'MA', 'display_address': ['4 Cambridge Ctr', 'Cambridge, MA 02142']}
Boston Common {'address1': 'Beacon St', 'address2': None, 'address3': '', 'city': 'Boston', 'zip_code': '02139', 'country': 'US', 'state': 'MA', 'display_address': ['Beacon St', 'Boston, MA 02139']}
The Esplanade {'address1': 'Storrow Dr', 'address2': '', 'address3': '', 'city': 'Boston', 'zip_code': '02116', 'country': 'US', 'state': 'MA', 'display_address': ['Storrow Dr', 'Boston, MA 02116']}
Community Ice Skating at Kendall Square {'address1': '300 Athenaeum St', 'address2': None, 'address3': '', 'city': 'Cambridge', 'zip_code': '02142', 'country': 'US', 'state': 'MA', 'display_address': ['300 Athenaeum St', 'Cambridge, MA 02142']}
Trapology Boston {'address1': '177 Tremont St', 'address2': 'Fl 2', 'address3': '', 'city': 'Boston', 'zip_code': '02111', 'cou

In [7]:
conn = sqlite3.connect('yelp.db')
cur = conn.cursor()

In [8]:
# normalizing json to pandas dataframe
df = json_normalize(results)

In [9]:
df

,categories,coordinates.latitude,coordinates.longitude,display_phone,distance,id,image_url,is_closed,location.address1,location.address2,...,location.display_address,location.state,location.zip_code,name,phone,price,rating,review_count,transactions,url
0,"[{'alias': 'parks', 'title': 'Parks'}]",42.363166,-71.086729,,233.226556,cambridge-center-roof-garden-cambridge,https://s3-media2.fl.yelpcdn.com/bphoto/4znlgU...,False,4 Cambridge Ctr,,...,"[4 Cambridge Ctr, Cambridge, MA 02142]",MA,02142,Cambridge Center Roof Garden,,NaN,5.0,56,[],https://www.yelp.com/biz/cambridge-center-roof...
1,"[{'alias': 'parks', 'title': 'Parks'}]",42.355373,-71.065750,(304) 563-4951,1801.659734,boston-common-boston,https://s3-media1.fl.yelpcdn.com/bphoto/Jhx9n_...,False,Beacon St,None,...,"[Beacon St, Boston, MA 02139]",MA,02139,Boston Common,+13045634951,NaN,4.5,397,[],https://www.yelp.com/biz/boston-common-boston?...
2,"[{'alias': 'parks', 'title': 'Parks'}]",42.357735,-71.073371,(617) 727-9547,1138.832487,the-esplanade-boston-2,https://s3-media4.fl.yelpcdn.com/bphoto/4lKESt...,False,Storrow Dr,,...,"[Storrow Dr, Boston, MA 02116]",MA,02116,The Esplanade,+16177279547,NaN,4.5,85,[],https://www.yelp.com/biz/the-esplanade-boston-...
3,"[{'alias': 'skatingrinks', 'title': 'Skating R...",42.364580,-71.081500,(617) 492-0941,233.540483,community-ice-skating-at-kendall-square-cambridge,https://s3-media4.fl.yelpcdn.com/bphoto/QLdwoM...,False,300 Athenaeum St,None,...,"[300 Athenaeum St, Cambridge, MA 02142]",MA,02142,Community Ice Skating at Kendall Square,+16174920941,NaN,4.5,35,[],https://www.yelp.com/biz/community-ice-skating...
4,"[{'alias': 'escapegames', 'title': 'Escape Gam...",42.353014,-71.064216,(857) 285-2085,2055.066728,trapology-boston-boston,https://s3-media4.fl.yelpcdn.com/bphoto/s7_-qt...,False,177 Tremont St,Fl 2,...,"[177 Tremont St, Fl 2, Boston, MA 02111]",MA,02111,Trapology Boston,+18572852085,NaN,5.0,126,[],https://www.yelp.com/biz/trapology-boston-bost...
5,"[{'alias': 'pilates', 'title': 'Pilates'}, {'a...",42.352040,-71.072590,(617) 578-8663,1647.815650,btone-fitness-back-bay-boston,https://s3-media4.fl.yelpcdn.com/bphoto/I3gcI1...,False,30 Newbury St,Fl 4,...,"[30 Newbury St, Fl 4, Boston, MA 02116]",MA,02116,Btone Fitness - Back Bay,+16175788663,NaN,4.5,183,[],https://www.yelp.com/biz/btone-fitness-back-ba...
6,"[{'alias': 'parks', 'title': 'Parks'}]",42.360307,-71.072563,,1046.413810,charles-river-bike-paths-boston,https://s3-media2.fl.yelpcdn.com/bphoto/HrngKN...,False,Storrow And Memorial Dr,None,...,"[Storrow And Memorial Dr, Boston, MA 02228]",MA,02228,Charles River Bike Paths,,NaN,4.5,77,[],https://www.yelp.com/biz/charles-river-bike-pa...
7,"[{'alias': 'active', 'title': 'Active Life'}, ...",42.361275,-71.074905,,827.063258,the-charles-river-boston,https://s3-media4.fl.yelpcdn.com/bphoto/9-DD7M...,False,Storrow Dr,None,...,"[Storrow Dr, Boston, MA 02114]",MA,02114,The Charles River,,NaN,4.5,42,[],https://www.yelp.com/biz/the-charles-river-bos...
8,"[{'alias': 'gyms', 'title': 'Gyms'}, {'alias':...",42.363680,-71.083410,(617) 252-0020,81.803994,bodyscapes-fitness-cambridge,https://s3-media2.fl.yelpcdn.com/bphoto/Xt74Ot...,False,356 Third St,,...,"[356 Third St, Cambridge, MA 02142]",MA,02142,BodyScapes Fitness,+16172520020,NaN,4.5,26,[],https://www.yelp.com/biz/bodyscapes-fitness-ca...
9,"[{'alias': 'cyclingclasses', 'title': 'Cycling...",42.366001,-71.091158,(857) 242-3039,608.296732,turnstyle-kendall-square-cambridge-2,https://s3-media3.fl.yelpcdn.com/bphoto/J1k6jd...,False,One Kendall Square,Ste B3002,...,"[One Kendall Square, Ste B3002, Cambridge, MA ...",MA,02139,Turnstyle - Kendall Square,+18572423039,NaN,4.0,65,[],https://www.yelp.com/biz/turnstyle-kendall-squ...


In [10]:
df.dtypes

categories                   object
coordinates.latitude        float64
coordinates.longitude       float64
display_phone                object
distance                    float64
id                           object
image_url                    object
is_closed                      bool
location.address1            object
location.address2            object
location.address3            object
location.city                object
location.country             object
location.display_address     object
location.state               object
location.zip_code            object
name                         object
phone                        object
price                        object
rating                      float64
review_count                  int64
transactions                 object
url                          object
dtype: object

In [11]:
df = df.drop(['categories', 'location.display_address', 'transactions'], axis=1)

In [12]:
df.columns = df.columns.str.replace(r'[.]', '_')

In [13]:
df.columns

Index(['coordinates_latitude', 'coordinates_longitude', 'display_phone',
       'distance', 'id', 'image_url', 'is_closed', 'location_address1',
       'location_address2', 'location_address3', 'location_city',
       'location_country', 'location_state', 'location_zip_code', 'name',
       'phone', 'price', 'rating', 'review_count', 'url'],
      dtype='object')

In [14]:
# converting to sqlite
df.to_sql("yelp_businesses", conn, if_exists="replace")

In [15]:
pd.read_sql_query("select * from yelp_businesses;", conn)

,index,coordinates_latitude,coordinates_longitude,display_phone,distance,id,image_url,is_closed,location_address1,location_address2,...,location_city,location_country,location_state,location_zip_code,name,phone,price,rating,review_count,url
0,0,42.363166,-71.086729,,233.226556,cambridge-center-roof-garden-cambridge,https://s3-media2.fl.yelpcdn.com/bphoto/4znlgU...,0,4 Cambridge Ctr,,...,Cambridge,US,MA,02142,Cambridge Center Roof Garden,,None,5.0,56,https://www.yelp.com/biz/cambridge-center-roof...
1,1,42.355373,-71.065750,(304) 563-4951,1801.659734,boston-common-boston,https://s3-media1.fl.yelpcdn.com/bphoto/Jhx9n_...,0,Beacon St,None,...,Boston,US,MA,02139,Boston Common,+13045634951,None,4.5,397,https://www.yelp.com/biz/boston-common-boston?...
2,2,42.357735,-71.073371,(617) 727-9547,1138.832487,the-esplanade-boston-2,https://s3-media4.fl.yelpcdn.com/bphoto/4lKESt...,0,Storrow Dr,,...,Boston,US,MA,02116,The Esplanade,+16177279547,None,4.5,85,https://www.yelp.com/biz/the-esplanade-boston-...
3,3,42.364580,-71.081500,(617) 492-0941,233.540483,community-ice-skating-at-kendall-square-cambridge,https://s3-media4.fl.yelpcdn.com/bphoto/QLdwoM...,0,300 Athenaeum St,None,...,Cambridge,US,MA,02142,Community Ice Skating at Kendall Square,+16174920941,None,4.5,35,https://www.yelp.com/biz/community-ice-skating...
4,4,42.353014,-71.064216,(857) 285-2085,2055.066728,trapology-boston-boston,https://s3-media4.fl.yelpcdn.com/bphoto/s7_-qt...,0,177 Tremont St,Fl 2,...,Boston,US,MA,02111,Trapology Boston,+18572852085,None,5.0,126,https://www.yelp.com/biz/trapology-boston-bost...
5,5,42.352040,-71.072590,(617) 578-8663,1647.815650,btone-fitness-back-bay-boston,https://s3-media4.fl.yelpcdn.com/bphoto/I3gcI1...,0,30 Newbury St,Fl 4,...,Boston,US,MA,02116,Btone Fitness - Back Bay,+16175788663,None,4.5,183,https://www.yelp.com/biz/btone-fitness-back-ba...
6,6,42.360307,-71.072563,,1046.413810,charles-river-bike-paths-boston,https://s3-media2.fl.yelpcdn.com/bphoto/HrngKN...,0,Storrow And Memorial Dr,None,...,Boston,US,MA,02228,Charles River Bike Paths,,None,4.5,77,https://www.yelp.com/biz/charles-river-bike-pa...
7,7,42.361275,-71.074905,,827.063258,the-charles-river-boston,https://s3-media4.fl.yelpcdn.com/bphoto/9-DD7M...,0,Storrow Dr,None,...,Boston,US,MA,02114,The Charles River,,None,4.5,42,https://www.yelp.com/biz/the-charles-river-bos...
8,8,42.363680,-71.083410,(617) 252-0020,81.803994,bodyscapes-fitness-cambridge,https://s3-media2.fl.yelpcdn.com/bphoto/Xt74Ot...,0,356 Third St,,...,Cambridge,US,MA,02142,BodyScapes Fitness,+16172520020,None,4.5,26,https://www.yelp.com/biz/bodyscapes-fitness-ca...
9,9,42.366001,-71.091158,(857) 242-3039,608.296732,turnstyle-kendall-square-cambridge-2,https://s3-media3.fl.yelpcdn.com/bphoto/J1k6jd...,0,One Kendall Square,Ste B3002,...,Cambridge,US,MA,02139,Turnstyle - Kendall Square,+18572423039,None,4.0,65,https://www.yelp.com/biz/turnstyle-kendall-squ...


# Uber API

In [72]:
# pip install uber-rides
from uber_rides.session import Session
from uber_rides.client import UberRidesClient
# conda install -c conda-forge geopy
from geopy.distance import vincenty
import csv

session = Session(server_token='Uvu3eEPnLtPKCbTU7KrCko5jo1ua4CVgYAqd0JfO')
client = UberRidesClient(session)

In [26]:
df1 = pd.read_sql_query("SELECT name, coordinates_latitude, coordinates_longitude, location_address1, location_address2, location_address3, location_city, location_state, location_zip_code, location_country FROM yelp_businesses ORDER BY RANDOM() LIMIT 1;", conn)
df1.head()

,name,coordinates_latitude,coordinates_longitude,location_address1,location_address2,location_address3,location_city,location_state,location_zip_code,location_country
0,The Boston Common Frog Pond,42.355912,-71.065922,Beacon St,,,Boston,MA,02108,US


In [27]:
df2 = pd.read_sql_query('SELECT name, coordinates_latitude, coordinates_longitude, location_address1, location_address2, location_address3, location_city, location_state, location_zip_code, location_country FROM yelp_businesses where name IN (SELECT name FROM yelp_businesses ORDER BY RANDOM() LIMIT 1)', conn)
df2.head()

,name,coordinates_latitude,coordinates_longitude,location_address1,location_address2,location_address3,location_city,location_state,location_zip_code,location_country
0,Prana Power Yoga,42.365095,-71.10273,585 Mass Ave,,,Cambridge,MA,02139,US


In [28]:
start_loc = (df1['coordinates_latitude'][0], df1['coordinates_longitude'][0])
start_loc

(42.355911766808497, -71.065921783447294)

In [29]:
end_loc = (df2['coordinates_latitude'][0], df2['coordinates_longitude'][0])
end_loc

(42.365094999999997, -71.102729999999994)

In [39]:
distance = vincenty(start_loc, end_loc).miles
if(distance > 1):
    print(distance, '\n')
    response = client.get_price_estimates(
    start_latitude= df1['coordinates_latitude'][0],
    start_longitude= df1['coordinates_longitude'][0],
    end_latitude=  df2['coordinates_latitude'][0],
    end_longitude= df2['coordinates_longitude'][0]
    )
    uber_rides = response.json.get("prices")
    print(uber_rides)

1.9879500584043308 

[{'localized_display_name': 'POOL', 'distance': 2.47, 'display_name': 'POOL', 'product_id': '997acbb5-e102-41e1-b155-9df7de0a73f2', 'high_estimate': 12.0, 'low_estimate': 8.0, 'duration': 420, 'estimate': '$8-11', 'currency_code': 'USD'}, {'localized_display_name': 'uberX', 'distance': 2.47, 'display_name': 'uberX', 'product_id': '55c66225-fbe7-4fd5-9072-eab1ece5e23e', 'high_estimate': 11.0, 'low_estimate': 8.0, 'duration': 420, 'estimate': '$8-11', 'currency_code': 'USD'}, {'localized_display_name': 'uberSUV', 'distance': 2.47, 'display_name': 'uberSUV', 'product_id': '6d318bcc-22a3-4af6-bddd-b409bfce1546', 'high_estimate': 36.0, 'low_estimate': 28.0, 'duration': 420, 'estimate': '$28-36', 'currency_code': 'USD'}, {'localized_display_name': 'uberXL', 'distance': 2.47, 'display_name': 'uberXL', 'product_id': '6f72dfc5-27f1-42e8-84db-ccc7a75f6969', 'high_estimate': 17.0, 'low_estimate': 13.0, 'duration': 420, 'estimate': '$13-17', 'currency_code': 'USD'}, {'localize

In [40]:
    dt = datetime.datetime.now()    
    for rides in uber_rides:
        rides["time"] = dt.strftime('%H:%M:%S')
        rides['day'] = dt.strftime('%A')
        rides['date'] = dt.strftime('%B %d, %Y')
        rides["start_latitude"] = df1['coordinates_latitude'][0]
        rides["start_longitude"] = df1['coordinates_longitude'][0]
        rides["end_latitude"] = df2['coordinates_latitude'][0]
        rides["end_longitude"] = df2['coordinates_longitude'][0]
        rides['start_location'] = df1['name'][0]
        rides['end_location'] = df2['name'][0]

In [41]:
        df_uber = pd.DataFrame(uber_rides)
        df_uber

,currency_code,date,day,display_name,distance,duration,end_latitude,end_location,end_longitude,estimate,high_estimate,localized_display_name,low_estimate,product_id,start_latitude,start_location,start_longitude,time
0,USD,"February 22, 2018",Thursday,POOL,2.47,420,42.365095,Prana Power Yoga,-71.10273,$8-11,12.0,POOL,8.0,997acbb5-e102-41e1-b155-9df7de0a73f2,42.355912,The Boston Common Frog Pond,-71.065922,01:35:21
1,USD,"February 22, 2018",Thursday,uberX,2.47,420,42.365095,Prana Power Yoga,-71.10273,$8-11,11.0,uberX,8.0,55c66225-fbe7-4fd5-9072-eab1ece5e23e,42.355912,The Boston Common Frog Pond,-71.065922,01:35:21
2,USD,"February 22, 2018",Thursday,uberSUV,2.47,420,42.365095,Prana Power Yoga,-71.10273,$28-36,36.0,uberSUV,28.0,6d318bcc-22a3-4af6-bddd-b409bfce1546,42.355912,The Boston Common Frog Pond,-71.065922,01:35:21
3,USD,"February 22, 2018",Thursday,uberXL,2.47,420,42.365095,Prana Power Yoga,-71.10273,$13-17,17.0,uberXL,13.0,6f72dfc5-27f1-42e8-84db-ccc7a75f6969,42.355912,The Boston Common Frog Pond,-71.065922,01:35:21
4,USD,"February 22, 2018",Thursday,UberBLACK,2.47,420,42.365095,Prana Power Yoga,-71.10273,$20-25,25.0,UberBLACK,20.0,6c84fd89-3f11-4782-9b50-97c468b19529,42.355912,The Boston Common Frog Pond,-71.065922,01:35:21
5,USD,"February 22, 2018",Thursday,uberWAV,2.47,420,42.365095,Prana Power Yoga,-71.10273,$8-11,11.0,uberWAV,8.0,9a0e7b09-b92b-4c41-9779-2ad22b4d779d,42.355912,The Boston Common Frog Pond,-71.065922,01:35:21
6,None,"February 22, 2018",Thursday,TAXI,2.47,420,42.365095,Prana Power Yoga,-71.10273,Metered,NaN,TAXI,NaN,8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a,42.355912,The Boston Common Frog Pond,-71.065922,01:35:21


In [43]:
        file_name = os.path.join(os.getcwd(), 'uber.csv')
        df_uber.to_csv(file_name, sep=',', encoding='utf-8', index=False)

In [44]:
# to append when sending to server
with open('uber.csv', 'a') as f:
    df_uber.to_csv(f, sep=',', encoding='utf-8', index=False, header=False)

In [45]:
# to do: define function and iterate else statement to that function if geopy distance < 1

# Lyft API

In [46]:
from lyft_rides.auth import ClientCredentialGrant
from lyft_rides.session import Session
from lyft_rides.auth import AuthorizationCodeGrant

In [47]:
auth_flow = ClientCredentialGrant(
    'gRUenY4LPYg_',
    'dFyiT-f23Jwmo_A7n2xGfzp_WcWvBIi8',
    'public',
    )
lyft_session = auth_flow.get_session()

In [48]:
#Use the same location
df1.head()

,name,coordinates_latitude,coordinates_longitude,location_address1,location_address2,location_address3,location_city,location_state,location_zip_code,location_country
0,The Boston Common Frog Pond,42.355912,-71.065922,Beacon St,,,Boston,MA,02108,US


In [49]:
df2.head()

,name,coordinates_latitude,coordinates_longitude,location_address1,location_address2,location_address3,location_city,location_state,location_zip_code,location_country
0,Prana Power Yoga,42.365095,-71.10273,585 Mass Ave,,,Cambridge,MA,02139,US


In [50]:
#Check if it is the same location
#start_loc

In [51]:
#Check if it is the same location
#end_loc

Reference of lyft api: https://developer.lyft.com/v1/reference
<br>
Estimated upper bound for trip cost, in minor units (cents).

In [52]:
#Get the ride type, Introduction to different type of lyft:https://developer.lyft.com/docs/glossary
from lyft_rides.client import LyftRidesClient

lyft_client=LyftRidesClient(lyft_session)
lyft_type_response = lyft_client.get_ride_types(df1['coordinates_latitude'][0], df1['coordinates_longitude'][0])
ride_types = lyft_type_response.json.get('ride_types')
print(ride_types)

[{'ride_type': 'lyft_line', 'display_name': 'Lyft Line', 'image_url': 'https://cdn.lyft.com/assets/car_standard.png', 'pricing_details': {'base_charge': 200, 'cost_per_mile': 129, 'cost_per_minute': 20, 'cost_minimum': 350, 'trust_and_service': 160, 'currency': 'USD', 'cancel_penalty_amount': 500}, 'seats': 2}, {'ride_type': 'lyft', 'display_name': 'Lyft', 'image_url': 'https://cdn.lyft.com/assets/car_standard.png', 'pricing_details': {'base_charge': 200, 'cost_per_mile': 129, 'cost_per_minute': 20, 'cost_minimum': 500, 'trust_and_service': 160, 'currency': 'USD', 'cancel_penalty_amount': 500}, 'seats': 4}, {'ride_type': 'lyft_plus', 'display_name': 'Lyft Plus', 'image_url': 'https://cdn.lyft.com/assets/car_plus.png', 'pricing_details': {'base_charge': 350, 'cost_per_mile': 250, 'cost_per_minute': 35, 'cost_minimum': 600, 'trust_and_service': 185, 'currency': 'USD', 'cancel_penalty_amount': 500}, 'seats': 6}, {'ride_type': 'lyft_premier', 'display_name': 'Lyft Premier', 'image_url': 'h

In [53]:
#Get the ride estimated cost
distance = vincenty(start_loc, end_loc).miles
if(distance > 1):
    print(distance)
    lyft_price_response = lyft_client.get_cost_estimates(
    start_latitude= df1['coordinates_latitude'][0],
    start_longitude= df1['coordinates_longitude'][0],
    end_latitude=  df2['coordinates_latitude'][0],
    end_longitude= df2['coordinates_longitude'][0]
    )
    lyft_rides = lyft_price_response.json.get('cost_estimates')
    print(lyft_rides)

1.9879500584043308
[{'ride_type': 'lyft_premier', 'estimated_duration_seconds': 611, 'estimated_distance_miles': 2.75, 'estimated_cost_cents_max': 1787, 'primetime_percentage': '0%', 'is_valid_estimate': True, 'currency': 'USD', 'cost_token': None, 'estimated_cost_cents_min': 1787, 'display_name': 'Lyft Premier', 'primetime_confirmation_token': None, 'can_request_ride': True}, {'ride_type': 'lyft_line', 'estimated_duration_seconds': 611, 'estimated_distance_miles': 2.75, 'estimated_cost_cents_max': 721, 'primetime_percentage': '0%', 'is_valid_estimate': True, 'currency': 'USD', 'cost_token': None, 'estimated_cost_cents_min': 721, 'display_name': 'Lyft Line', 'primetime_confirmation_token': None, 'can_request_ride': True}, {'ride_type': 'lyft_lux', 'estimated_duration_seconds': 611, 'estimated_distance_miles': 2.75, 'estimated_cost_cents_max': 2329, 'primetime_percentage': '0%', 'is_valid_estimate': True, 'currency': 'USD', 'cost_token': None, 'estimated_cost_cents_min': 2329, 'display_

In [55]:
for rides in lyft_rides:
    rides["time"] = dt.strftime('%H:%M:%S')
    rides['day'] = dt.strftime('%A')
    rides['date'] = dt.strftime('%B %d, %Y')
    rides["start_latitude"] = df1['coordinates_latitude'][0]
    rides["start_longitude"] = df1['coordinates_longitude'][0]
    rides["end_latitude"] = df2['coordinates_latitude'][0]
    rides["end_longitude"] = df2['coordinates_longitude'][0]
    rides['start_location'] = df1['name'][0]
    rides['end_location'] = df2['name'][0]

In [56]:
df_lyft = pd.DataFrame(lyft_rides)
df_lyft

,can_request_ride,cost_token,currency,date,day,display_name,end_latitude,end_location,end_longitude,estimated_cost_cents_max,...,estimated_distance_miles,estimated_duration_seconds,is_valid_estimate,primetime_confirmation_token,primetime_percentage,ride_type,start_latitude,start_location,start_longitude,time
0,True,None,USD,"February 22, 2018",Thursday,Lyft Premier,42.365095,Prana Power Yoga,-71.10273,1787,...,2.75,611,True,None,0%,lyft_premier,42.355912,The Boston Common Frog Pond,-71.065922,01:35:21
1,True,None,USD,"February 22, 2018",Thursday,Lyft Line,42.365095,Prana Power Yoga,-71.10273,721,...,2.75,611,True,None,0%,lyft_line,42.355912,The Boston Common Frog Pond,-71.065922,01:35:21
2,True,None,USD,"February 22, 2018",Thursday,Lyft Lux,42.365095,Prana Power Yoga,-71.10273,2329,...,2.75,611,True,None,0%,lyft_lux,42.355912,The Boston Common Frog Pond,-71.065922,01:35:21
3,True,None,USD,"February 22, 2018",Thursday,Lyft,42.365095,Prana Power Yoga,-71.10273,934,...,2.75,611,True,None,0%,lyft,42.355912,The Boston Common Frog Pond,-71.065922,01:35:21
4,True,None,USD,"February 22, 2018",Thursday,Lyft Plus,42.365095,Prana Power Yoga,-71.10273,1540,...,2.75,611,True,None,0%,lyft_plus,42.355912,The Boston Common Frog Pond,-71.065922,01:35:21
5,True,None,USD,"February 22, 2018",Thursday,Lyft Lux SUV,42.365095,Prana Power Yoga,-71.10273,3151,...,2.75,611,True,None,0%,lyft_luxsuv,42.355912,The Boston Common Frog Pond,-71.065922,01:35:21


In [57]:
file_name_lyft = os.path.join(os.getcwd(), 'lyft.csv')
df_lyft.to_csv(file_name_lyft, sep=',', encoding='utf-8', index=False)

In [58]:
# to append when sending to server
with open('lyft.csv', 'a') as f:
    df_lyft.to_csv(f, sep=',', encoding='utf-8', index=False, header=False)

# Weather API

In [59]:
import requests
import time

In [62]:
#Get the current weather information from lat and long
#http://api.openweathermap.org/data/2.5/weather?lat=42.37046&lon=-71.10352&appid=119fe664452f079528a64467c793dd7d
lat=str(df1['coordinates_latitude'][0])
long=str(df1['coordinates_longitude'][0])
api_address='http://api.openweathermap.org/data/2.5/weather?lat='+lat+'&lon='+long+'&appid=119fe664452f079528a64467c793dd7d&q='

# real_time=time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

weather_data = requests.get(api_address).json()

print(weather_data)
# print(real_time)

{'coord': {'lon': -71.07, 'lat': 42.36}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 284.9, 'pressure': 1031, 'humidity': 40, 'temp_min': 283.15, 'temp_max': 286.15}, 'visibility': 16093, 'wind': {'speed': 7.7, 'deg': 330, 'gust': 11.3}, 'clouds': {'all': 90}, 'dt': 1519280100, 'sys': {'type': 1, 'id': 1274, 'message': 0.0047, 'country': 'US', 'sunrise': 1519299010, 'sunset': 1519338351}, 'id': 4930956, 'name': 'Boston', 'cod': 200}


In [65]:
df_weather = json_normalize(weather_data)
df_weather.head()

,base,clouds.all,cod,coord.lat,coord.lon,dt,id,main.humidity,main.pressure,main.temp,...,sys.id,sys.message,sys.sunrise,sys.sunset,sys.type,visibility,weather,wind.deg,wind.gust,wind.speed
0,stations,90,200,42.36,-71.07,1519280100,4930956,40,1031,284.9,...,1274,0.0047,1519299010,1519338351,1,16093,"[{'id': 804, 'main': 'Clouds', 'description': ...",330,11.3,7.7


In [66]:
df_weather.columns

Index(['base', 'clouds.all', 'cod', 'coord.lat', 'coord.lon', 'dt', 'id',
       'main.humidity', 'main.pressure', 'main.temp', 'main.temp_max',
       'main.temp_min', 'name', 'sys.country', 'sys.id', 'sys.message',
       'sys.sunrise', 'sys.sunset', 'sys.type', 'visibility', 'weather',
       'wind.deg', 'wind.gust', 'wind.speed'],
      dtype='object')

In [69]:
df_weather.columns = df_weather.columns.str.replace(r'[.]', '_')
df_weather.head()

,base,clouds_all,cod,coord_lat,coord_lon,dt,id,main_humidity,main_pressure,main_temp,...,sys_id,sys_message,sys_sunrise,sys_sunset,sys_type,visibility,weather,wind_deg,wind_gust,wind_speed
0,stations,90,200,42.36,-71.07,1519280100,4930956,40,1031,284.9,...,1274,0.0047,1519299010,1519338351,1,16093,"[{'id': 804, 'main': 'Clouds', 'description': ...",330,11.3,7.7


In [63]:
# Get weather status of the weather information:
# status = weather_data['weather'][0]['main']
# print(status)

NameError: name 'json_data' is not defined

In [71]:
# set up Ubuntu server
# create user and add user to a group
# tweaked user privileges
# change server's timezone to Eastern (for our datetime purposes)
# install packages
# history on command prompt
# install anaconda environment (curl -O https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh)
# scp our sqlite database file to server so the script can skip querying yelp's API
# we can add more locations to yelp database by querying 50 in the same zipcode, increasing our db size by 50 times
# set up cron job at minute intervals